In [1]:
import os
import sys
module_path = "/data/luojaa/eukgen/"
sys.path.append(module_path)
from core_functions.helper_functions import fasta_to_dict, dict_to_fasta, filter_by_entropy 
import pandas as pd
import numpy as np

In [2]:
cd /data/luojaa/eukgen/

/vf/users/luojaa/eukgen


## build profiles from repclade usalign

In [3]:
fasta_root = "/data/luojaa/polvir/USalignDB/repclade.usalign_clean.entropyfilt.fasta"
output_root = "/data/luojaa/polvir/processing/hhdb/"
DB_name = "polcore"
cmd_hhmake = f"hhmake -i {fasta_root} -o {output_root}{DB_name}.hmm -M 50 -add_cons"
#os.system(cmd_hhmake)

## hhalign search against all seqs

In [4]:
#bash command from /data/luojaa/polvir/seqs
#for f in *; do hhalign -i $f -t ../processing/hhdb/polcore.hmm -o ../processing/hhalign_out/$f.ffdata; done

## Parse hhalign output 

In [72]:
from core_functions.hhsuite_functions import merge_hhsuite_search_results

In [73]:
search_root= "/data/luojaa/polvir/processing/hhalign_out/"
output_name = "/data/luojaa/polvir/processing/merged_hhalign"
chunk_no = 10
pairwise_cov = .1
probability = .1

In [75]:
#merge_hhsuite_search_results(search_root, output_name, write_tsv=True, filter_cov=pairwise_cov, filter_prob=probability)

## tune alignment confidences and splice out larger insertions

In [76]:
from Bio import PDB

In [77]:
hhalign_merged = "/data/luojaa/polvir/processing/merged_hhalign.tsv"
hhalign_raw = pd.read_csv(hhalign_merged, sep = "\t")
hhalign_df = hhalign_raw[hhalign_raw.Query != hhalign_raw.Target].reset_index()

In [85]:
def conf_encode(scores, target_seq):
    arr_score, rv = np.array(list(scores)), np.array(list(scores))
    arr_seq = np.array(list(target_seq))
    querygap = list((arr_seq != "-") & (arr_score == " "))
    targetgap = list(arr_seq == "-")
    for i, deletion in enumerate(querygap):
        if deletion:
            rv[i] = '7'
    for i, insertion in enumerate(targetgap):
        if insertion:
            rv[i] = '0'
    return [int(i) > 5 for i in rv]

In [86]:
hhalign_df["conf_encoded"] = hhalign_df.apply(lambda x: conf_encode(x.Confidence, x.Target_sequence), axis = 1)

In [88]:
def gap_bounds(bin_array):
    # Convert the input to a NumPy array if it's not already
    arr = np.array(bin_array, dtype=bool)
    
    # Find runs of False
    runs = np.concatenate(([0], np.where(np.diff(arr))[0] + 1, [len(arr)]))
    lengths = np.diff(runs)
    
    # Find long rungs
    long_runs = np.where((lengths > 30) & (~arr[runs[:-1]]))[0]
    
    # Get start and end indices of long False runs
    result = [(runs[i], runs[i+1]-1) for i in long_runs]
    
    return result

In [103]:
hhalign_df["gap_bounds_inplace"] = hhalign_df.conf_encoded.apply(lambda x: gap_bounds(x))
hhalign_df["num_gaps"] = hhalign_df.gap_bounds_inplace.apply(lambda x: len(x))
#hhalign_df["gap_bounds_offset"] = hhalign_df.apply(lambda x: [(t[0] + x['Query-HMM-start'], t[1] + x['Query-HMM-start']) for t in x["gap_bounds_inplace"]], axis = 1)

In [125]:
hhalign_df.iloc[-1, -3]

[(121, 167),
 (246, 281),
 (339, 399),
 (443, 530),
 (665, 708),
 (710, 793),
 (812, 851),
 (927, 1014),
 (1033, 1106)]

In [119]:
hhalign_df.iloc[-1, -12]

70

In [131]:
"RG-------LHFSSIKIVKLFKLHGFQK-EKSSYIRVQFNNLTDRKKCIEYVDRMNTELEMAGRRKMETASDDLGGGNYY"[33+7:]

'NLTDRKKCIEYVDRMNTELEMAGRRKMETASDDLGGGNYY'

In [123]:
def map_gaps(query_seq, query_offset):
    j = query_offset - 1
    gapmap = {}
    for i, aa in enumerate(list(query_seq)):
        if aa != "-":
            j += 1
        gapmap[i] = j
    return gapmap

gapmap_2 = map_gaps(hhalign_df.iloc[-1, -10], hhalign_df.iloc[-1, 12])

In [126]:
gapmap_2[121]

177

In [180]:
# def find_gaps(string):
#     gaps = []
#     in_gap = False
#     start = 0
#     for i, char in enumerate(string):
#         if char == "-":
#             if not in_gap:
#                 start = i
#                 in_gap = True
#         else:
#             if in_gap:
#                 gaps.append((start, i))
#                 in_gap = False
#     if in_gap:
#         gaps.append((start, len(string)))
#     return gaps

In [179]:
# hhalign_df["gaps_raw"] = hhalign_df.Query_sequence.apply(lambda x: find_gaps(x))

In [189]:
def adjust_ranges(gaps_offset, query_str):
    gaps = []
    for t in gaps_offset:
        start, end = t
        start_offset = start - query_str[:start].count("-")
        end_offset = end - query_str[:end].count("-")
        gaps.append((start_offset, end_offset))
    return gaps

In [190]:
hhalign_df["gap_bounds_adj"] = hhalign_df.apply(lambda x: adjust_ranges(x["gap_bounds_offset"],x["Query_sequence"]), axis = 1)

,index,Query,Target,Prob,E-value,P-value,Score,SS,Cols,Identities,...,Matches,Target_consensus,Target_sequence,Confidence,conf_encoded,gap_bounds_inplace,num_gaps,gap_bounds_offset,true_gaps,gap_bounds_adj
0,1,A3_AE009439.1_1100,ED_10228.pdb,100.0,4.100000e-72,4.100000e-72,572.2,0.0,762.0,29,...,.|-+-|.|-.++||...+-|.||-||||.++..+|++.....+......,~piiRlfGVT~~G~SV~~~Vhgf~PYfYv~~p~~~~~~~~~~~~~~...,VPIIRMFGVTKEQHTVICHIHGFAPYFYVEVPPTCQREECQLFQDD...,3455578999999999999999999999999999988777665433...,"[False, False, False, False, False, True, True...","[(42, 83), (159, 198)]",2,"[(56, 97), (173, 212)]","[(56, 80), (112, 117), (118, 119), (159, 161),...","[(56, 73), (135, 163)]"
1,3,A3_AQRS01000042.1_3,ED_10228.pdb,100.0,2.400000e-50,2.400000e-50,401.4,0.0,764.0,25,...,+.=.++|+||...+- -.+.|++|--|+ ..-|.+.-|.|...,~~fq~ldid~~~~~~~~~~~~~~~piiRlfGVT~~G~SV~~~Vhgf...,IIFQQLDMDYYLGSPVP-M-TGPVPIIRMFGVTKEQHTVICHIHGF...,344678999988762 345689998885 77899999999...,"[False, False, False, True, True, True, True, ...","[(55, 108), (168, 207), (316, 349)]",3,"[(61, 114), (174, 213), (322, 355)]","[(15, 22), (34, 35), (56, 58), (67, 71), (74, ...","[(51, 85), (139, 156), (253, 272)]"
2,5,A3_BDTY01000048.1_2,ED_10228.pdb,100.0,4.700000e-77,4.700000e-77,608.6,0.0,765.0,30,...,-|++||||-+.. .-.+||+|--.+.|.-.+..-++|+|...,fq~ldid~~~~~~~~~~~~~~~piiRlfGVT~~G~SV~~~Vhgf~P...,FQQLDMDYYLGSPVP-M-TGPVPIIRMFGVTKEQHTVICHIHGFAP...,368999998775 56899999999999999999999999...,"[False, True, True, True, True, True, True, Tr...","[(52, 84), (167, 202), (435, 466)]",3,"[(55, 87), (170, 205), (438, 469)]","[(12, 20), (53, 54), (65, 66), (70, 82), (95, ...","[(46, 65), (137, 154), (365, 394)]"
3,7,A3_LNJC01000017.1_12,ED_10228.pdb,100.0,5.700000e-71,5.700000e-71,560.8,0.0,733.0,26,...,...+-..-..|+||.|+-|+. . ..+.-+ +|-+.++... ...,G~SV~~~Vhgf~PYfYv~~p~~~~~~~~~~~~~~l~~~l~~~~~~~...,QHTVICHIHGFAPYFYVEVPPTCQREECQLFQDDLNKTVLNDFKSK...,445667778999999999983 2 222222 2222332211 ...,"[False, False, False, True, True, True, True, ...","[(20, 69), (126, 175), (384, 427)]",3,"[(44, 93), (150, 199), (408, 451)]","[(21, 22), (23, 26), (32, 33), (43, 46), (48, ...","[(38, 80), (124, 155), (337, 379)]"
4,9,A3_MNVE01000009.1_147,ED_10228.pdb,100.0,4.000000e-57,4.000000e-57,454.5,0.0,728.0,26,...,-.||+||||..+|. ..+.++..+...+.... ...++. ...,Vhgf~PYfYv~~p~~~~~~~~~~~~~~l~~~l~~~~~~~~~~~~~v...,IHGFAPYFYVEVPPTCQREECQLFQDDLNKTVLNDFKSK-KVSQAI...,34699999999994 24455554444333221 111122 ...,"[False, False, True, True, True, True, True, T...","[(13, 62), (120, 167)]",2,"[(54, 103), (161, 208)]","[(14, 18), (35, 39), (45, 46), (50, 52), (58, ...","[(43, 79), (118, 156)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,769,Pokke_GVMAG-S-3300009702-144.fna,ED_10228.pdb,99.5,4.400000e-22,4.400000e-22,192.0,0.0,598.0,20,...,+--|-+.|+.-+|..+-+-+-+..|||-+..|..--.. ...,~piiRlfGVT~~G~SV~~~Vhgf~PYfYv~~p~~~~~~--------...,VPIIRMFGVTKEQHTVICHIHGFAPYFYVEVPPTCQRE--------...,34466889999999999999999999999888321111 ...,"[False, False, False, True, True, True, True, ...","[(32, 85), (104, 152), (160, 207), (318, 360),...",9,"[(100, 153), (172, 220), (228, 275), (386, 428...","[(66, 71), (78, 79), (93, 94), (120, 128), (16...","[(93, 138), (152, 199), (207, 254), (361, 403)..."
383,771,Pokke_SRX319064.32.dc.fa,ED_10228.pdb,99.7,4.200000e-25,4.200000e-25,215.9,0.0,608.0,20,...,.+.|..-|+++-....- --+.+-|+|-.|.....-+.|...,~~~fq~ldid~~~~~~~~~~~~~~~piiRlfGVT~~G~SV~~~Vhg...,AIIFQQLDMDYYLGSPVP-M-TGPVPIIRMFGVTKEQHTVICHIHG...,45566666666555431 234578999999999999999...,"[False, False, False, True, True, True, True, ...","[(56, 88), (116, 181), (192, 224), (328, 375),...",7,"[(102, 134), (162, 227), (238, 270), (374, 421...","[(17, 25), (56, 57), (59, 61), (77, 82), (84, ...","[(84, 113), (135, 196), (207, 239), (342, 389)..."
384,773,Pokke_SRX327722.61.dc.fa,ED_10228.pdb,100.0,1.400000e-38,1.400000e-38,319.2,0.0,816.0,23,...,..|+-|.+..-+.. ..|..--|.||||--.|-.+.+.|...,~~~~~fq~ldid~~~~~~~~~~~~~~

In [142]:
hhalign_df.iloc[0,-3]

[(42, 83), (159, 198)]

In [144]:
hhalign_df.iloc[0,-5][40:85]

'66543322  222211                        12246'

In [147]:
len("PYGGDRVELLRIVATYPKVVPKLREQVKKLDG-----V-KEVYEADIPFVRRAAVDLNLPPASEVDVSDLDTGSWSGLP-")

80

In [139]:
input_file = "/data/luojaa/polvir/af2_structures/A3_AE009439.pdb"
output_file = "/data/luojaa/polvir/processing/A3_AE009439.test.pdb"
omit_regions = hhalign_df.iloc[0,-1]

parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure("protein", input_file)

# Create a new structure for the spliced portion
new_structure = PDB.Structure.Structure("spliced_protein")

# Iterate through models, chains, and residues
for model in structure:
    new_model = PDB.Model.Model(model.id)
    new_structure.add(new_model)
    for chain in model:
        new_chain = PDB.Chain.Chain(chain.id)
        new_model.add(new_chain)
        for residue in chain:
            residue_num = residue.id[1]
            if not any(start <= residue_num <= end for start, end in omit_regions):
                new_chain.add(residue)

# Save the new structure
io = PDB.PDBIO()
io.set_structure(new_structure)
io.save(output_file)